In [2]:
import private_info.db_info as dbi
import mysql.connector
import pandas_datareader as web
import math
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
import requests as req
import datetime as dt
import pytz 
import csv
import os

In [3]:
def connect_to_db ():
    db = mysql.connector.connect(
        host = dbi.hostname,  
        user = dbi.username,    
        password = dbi.password,
        database = dbi.database,
        auth_plugin='mysql_native_password'
    )

    print ()
    print (db)

    mycursor = db.cursor()
    
    return db, mycursor

In [4]:
db_con, cur = connect_to_db ()

In [9]:
est = pytz.timezone('US/Eastern')
now = dt.datetime.now(tz=est)

time_parts = str(now).split(" ")
current_date = time_parts[0]
print(current_date)

2021-12-27


In [10]:
start_date = current_date
end_date = current_date

In [11]:
cur.execute('USE etf_pred;')
cur.execute('SHOW tables;')
res = cur.fetchall()

In [90]:
name_list = []
for each in res:
    if each[0] != 'fed_funds' and each[0] != 'fvx' and each[0] != 'tnx' and each[0] != 'vix':
         name_list.append(each[0])

In [95]:
asset_df_list = []

for each in name_list:
    
    asset_df_list.append( web.DataReader(each, data_source='yahoo', start = start_date, end = end_date) )

In [96]:
inc = 0
for each in asset_df_list:
    each['Date'] = each.index
    each.reset_index(drop=True, inplace=True)
    each = each[['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']]
    if each.shape[0] > 1:
        each.drop(each.tail(1).index,inplace=True)
        
    date_dt = each.iloc[0]['Date']
    
    cur.execute(f"SELECT Date FROM {name_list[inc]} ORDER BY Date DESC LIMIT 1;")
    prev_date = cur.fetchall()[0][0]
    prev_date_dt = dt.datetime.strptime(prev_date, '%Y-%m-%d')
    
    
    if date_dt > prev_date_dt:
    
        date = str(date_dt)[0:10]
        print(date)
        high = each.iloc[0]['High']
        low = each.iloc[0]['Low']
        open = each.iloc[0]['Open']
        close = each.iloc[0]['Close']
        volume = each.iloc[0]['Volume']
        adj_close = each.iloc[0]['Adj Close']
        
        cmd1 = f"INSERT INTO {name_list[inc]} (`Date`, `High`, `Low`, `Open`, `Close`, `Volume`, `Adj Close`) "
        cmd2 = f"VALUES ('{date}', {high}, {low}, {open}, {close}, {volume}, {adj_close})"
        cmd = cmd1 + cmd2
        
        print (cmd)
        
        cur.execute(cmd)
        db_con.commit()
    
    inc = inc + 1

In [7]:
delta = dt.timedelta(1)
temp_date = now - delta
temp_date = str(temp_date)[0:10]


fed_fund_url = f"https://markets.newyorkfed.org/read?productCode=50&eventCodes=500&limit=5&startPosition=0&sort=postDt:-1&format=csv"

In [8]:
fed_funds_data = req.get(fed_fund_url)
decoded_content = fed_funds_data.content.decode('utf-8')

print (decoded_content)

# cr = csv.reader(decoded_content.splitlines(), delimiter=',')
# fed_funds_list = list(cr)
# fed_funds_list.append([temp_date, None])

Effective Date,Rate Type,Rate (%),1st Percentile (%),25th Percentile (%),75th Percentile (%),99th Percentile (%),Volume ($Billions),Target Rate From (%),Target Rate To (%),Intra Day - Low (%),Intra Day - High (%),Standard Deviation (%),30-Day Average SOFR,90-Day Average SOFR,180-Day Average SOFR,SOFR Index,Revision Indicator (Y/N),Footnote ID
12/24/2021,EFFR,0.08,0.06,0.07,0.08,0.20,69,0.00,0.25,,,,,,,,,
12/23/2021,EFFR,0.08,0.06,0.07,0.09,0.20,71,0.00,0.25,,,,,,,,,
12/22/2021,EFFR,0.08,0.06,0.07,0.08,0.20,69,0.00,0.25,,,,,,,,,
12/21/2021,EFFR,0.08,0.06,0.07,0.08,0.20,69,0.00,0.25,,,,,,,,,
12/20/2021,EFFR,0.08,0.06,0.07,0.08,0.20,72,0.00,0.25,,,,,,,,,




In [45]:
fed_funds_df = pd.DataFrame(fed_funds_list)
fed_funds_df = fed_funds_df.rename(columns={0: 'Date', 1: 'DFF'})
fed_funds_df.drop(index=fed_funds_df.index[0], axis=0, inplace=True)
fed_funds_df.reindex()
fed_funds_df

,Date,DFF
1,1954-07-01,1.13
2,1954-07-02,1.250
3,1954-07-03,1.250
4,1954-07-04,1.250
5,1954-07-05,0.88
...,...,...
24646,2021-12-21,0.08
24647,2021-12-22,0.08
24648,2021-12-23,0.08
24649,2021-12-24,0.08
